<a href="https://colab.research.google.com/github/RogovanovDenis/autumn_python_2025/blob/main/RAG_lesson2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Устанавливаем необходимые зависимости
!pip install docling[all] -q
!pip install docling-core -q
# Установка зависимостей (для RAG, веб-скрапинга и Vosk)
!pip install openai pymupdf numpy requests beautifulsoup4 vosk pydub
# Скачиваем небольшую русскую модель Vosk для распознавания аудио
!wget -nc https://alphacephei.com/vosk/models/vosk-model-small-ru-0.22.zip
# Распаковываем и переименовываем папку для удобства
!unzip -n vosk-model-small-ru-0.22.zip
!mv vosk-model-small-ru-0.22 vosk-model-folder

# Импорт необходимых модулей
import os
import numpy as np
import requests
import json
import wave
from bs4 import BeautifulSoup
from io import BytesIO
from openai import OpenAI
from vosk import Model, KaldiRecognizer
from pydub import AudioSegment
from google.colab import files
from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import InputFormat

print("✅ Зависимости установлены.")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 105.8 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=652395ba6d024bd6bab6023696120ce2205eb141bfed4095b1a7d3bad3e3948b
  Stored in directory: /root/.cache/pip/wheels/7e/75/5b/e1d5c3756631e4bda806f6cc9640153b39484bb6f7b0b8def3
Successfully built srt
File ‘vosk-model-small-ru-0.22.zip’ already there; not retrieving.

Archive:  vosk-model-small-ru-0.22.zip
   creating: vosk-model-small-ru-0.22/
   creating: vosk-model-small-ru-0.22/graph/
   creating: vosk-model-small-ru-0.22/graph/phones/
  inflating: vosk-model-small-ru-0.22/graph/phones/word_boundary.int  
  inflating: vosk-model-small-ru-0.22/graph/Gr.fst  
  inflating: vosk-model-small-ru-0.22/graph/HCLr.fst  
  inflating: vosk-model-small-ru-0.22/graph/disambig_tid.int  
   creating: vosk-model-small-ru-0.22/am/
 

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


✅ Зависимости установлены.


In [3]:
# Блок загрузки для PDF
print("Начало загрузки файла...")
uploaded = files.upload()

# Берем первый файл
pdf_path = list(uploaded.keys())[0]

print(f"✅ Файл '{pdf_path}' успешно загружен.")

Начало загрузки файла...


Saving instrukciya-samsung-galaxy-s23-ultra-6.pdf to instrukciya-samsung-galaxy-s23-ultra-6.pdf
✅ Файл 'instrukciya-samsung-galaxy-s23-ultra-6.pdf' успешно загружен.


In [4]:
def parse_pdf_document(file_path):
    """Извлекает текст из PDF документа с помощью Docling."""
    try:
        # Создаем конвертер документа
        converter = DocumentConverter()

        # Загружаем и конвертируем документ
        result = converter.convert(file_path)

        # Извлекаем весь текст из документа
        full_document_text = result.document.export_to_markdown()

        return full_document_text
    except Exception as e:
        print(f"Ошибка при чтении PDF с помощью Docling: {e}")
        # Резервный вариант для простых PDF
        try:
            import fitz
            full_document_text = ""
            doc = fitz.open(file_path)
            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                full_document_text += page.get_text()
            doc.close()
            return full_document_text
        except Exception as e2:
            print(f"Резервный метод также не сработал: {e2}")
            return ""

document_text = parse_pdf_document(pdf_path)
print(f"Извлечено {len(document_text)} символов с помощью Docling.")

[INFO] 2025-12-02 15:58:36,280 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-02 15:58:36,294 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.4.0/torch/PP-OCRv4/det/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-02 15:58:38,590 [RapidOCR] download_file.py:82: Download size: 13.83MB
[INFO] 2025-12-02 15:58:38,865 [RapidOCR] download_file.py:95: Successfully saved to: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-02 15:58:38,867 [RapidOCR] torch.py:54: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-02 15:58:39,166 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-02 15:58:39,168 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.4.0/torch/PP-OCRv4/cls/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-02 15:58:40,151 [RapidOCR] downlo

Извлечено 281037 символов с помощью Docling.


In [5]:
def parse_website_data(url):
    """Извлекает данные с веб-сайта с помощью Docling."""
    try:
        # Создаем конвертер документа
        converter = DocumentConverter()

        # Указываем, что это веб-страница
        input_item = InputFormat(source=url, format="web_page")

        # Конвертируем веб-страницу
        result = converter.convert(input_item)

        # Извлекаем текст
        web_text = result.document.export_to_markdown()

        # Очищаем текст от лишних пробелов
        lines = [line.strip() for line in web_text.split('\n') if line.strip()]
        clean_text = '\n'.join(lines)

        return clean_text

    except Exception as e:
        print(f"Ошибка при парсинге сайта с помощью Docling: {e}")
        # Резервный вариант с BeautifulSoup
        try:
            response = requests.get(url, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Удаляем ненужные элементы
            for tag in soup(['script', 'style', 'nav', 'footer', 'header', 'aside']):
                tag.decompose()

            # Извлекаем текст из основных текстовых тегов
            texts = soup.find_all(['p', 'h1', 'h2', 'h3', 'li'])
            clean_text = "\n".join([tag.get_text().strip() for tag in texts if tag.get_text().strip()])

            return clean_text
        except requests.exceptions.RequestException as e:
            return f"[ОШИБКА ВЕБ-САЙТА: {e}]"
        except Exception as e2:
            return f"[ОБЩАЯ ОШИБКА ПАРСИНГА: {e2}]"

WEBSITE_URL = "https://habr.com/ru/news/968856/"

print(f"-> Обработка ВЕБ-САЙТА: {WEBSITE_URL}")
web_text = parse_website_data(WEBSITE_URL)

print("\n" + "*"*40)
print(f"ИЗВЛЕЧЕННЫЙ ТЕКСТ (первые 800 символов):\n{web_text[:800]}...")
print("*"*40)

-> Обработка ВЕБ-САЙТА: https://habr.com/ru/news/968856/
Ошибка при парсинге сайта с помощью Docling: EnumType.__call__() got an unexpected keyword argument 'source'

****************************************
ИЗВЛЕЧЕННЫЙ ТЕКСТ (первые 800 символов):
Figure AI завершила почти годичные испытания человекоподобных роботов Figure 02 на заводе BMW в Южной Каролине
Калифорнийский стартап Figure AI отчитался о завершении 11-месячных испытаний человекоподобных роботов Figure 02 (F.02) на заводе BMW в штате Южная Каролина. Стартап подвёл итоги развёртывания Figure 02 на фоне снятия модели с производства.
Пилотный проект был частью совместной работы компаний по испытанию гуманоидных роботов на настоящей сборочной линии. Figure AI сообщила, что её роботы помогли произвести 30 тыс. автомобилей BMW X3 и загрузить более 90 тыс. деталей из листового металла.
Генеральный директор стартапа Бретт Эдкок поделился видео с роботами, покрытыми царапинами, потёртостями и грязью, что наглядно демонстрирует рабо

In [6]:
def create_sliding_window_chunks(document_text, max_chars=1000, stride_overlap=200):
    """Разбивает текст на фрагменты, используя скользящее окно."""
    corpus_fragments = []
    # Шаг, на который сдвигается окно (Размер - Перекрытие)
    stride = max_chars - stride_overlap

    if stride <= 0:
        raise ValueError("Размер перекрытия должен быть меньше размера фрагмента.")

    # Создаём последовательность начальных позиций для разрезания текста с шагом stride.
    for start_index in range(0, len(document_text), stride):
        # Окно заканчивается на start_index + max_chars
        end_index = min(start_index + max_chars, len(document_text))
        fragment = document_text[start_index:end_index]
        corpus_fragments.append(fragment)

        # Если конец фрагмента совпал с концом текста, останавливаемся
        if end_index == len(document_text):
            break

    return corpus_fragments

In [8]:
# PDF-документ
corpus_fragments_pdf = create_sliding_window_chunks(document_text, max_chars=1000, stride_overlap=200)
print(f"PDF документ разбит на {len(corpus_fragments_pdf)} фрагментов для индексации.")

# Сайт
corpus_fragments_web = create_sliding_window_chunks(web_text, max_chars=250, stride_overlap=50)
print(f"Веб-страница разбита на {len(corpus_fragments_web)} фрагментов для индексации.")

PDF документ разбит на 352 фрагментов для индексации.
Веб-страница разбита на 13 фрагментов для индексации.


In [9]:
# 🔑 Установка API ключа. Получите его в Nebius Studio.
os.environ["OPENAI_API_KEY"] = "v1.CmQKHHN0YXRpY2tleS1lMDByZ2Vmd3o1Z2RtM2phODASIXNlcnZpY2VhY2NvdW50LWUwMG1tNThhNzR2Nndnc2tqMzIMCN2nvMkGENXahroDOgwI3arUlAcQgLzwywJAAloDZTAw.AAAAAAAAAAF-nYvmZXEy8pKTiL6qtDoSttyuUI8GtFy-IN2Orbj1TyDKZURF2TxkUdhLcSYfGBJGchi0Np5b5GlpKg__iQ4K"

# Настройка клиента для Nebius AI (совместим с OpenAI API)
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")
)

def create_embeddings(text_list, model="BAAI/bge-multilingual-gemma2"):
    """Вызывает API для преобразования списка текстов в список векторов (эмбеддингов)."""
    # Вызов метода embeddings.create для получения векторов
    response = client.embeddings.create(input=text_list, model=model)
    return response.data

In [10]:
# PDF-документ
# Индексация: Генерация векторов для всех фрагментов корпуса
print("⏳ Генерация эмбеддингов для PDF корпуса...")
fragment_embeddings_pdf = create_embeddings(corpus_fragments_pdf)
print(f"✅ Индексация PDF завершена. Получено {len(fragment_embeddings_pdf)} векторов.")

# Сайт
# Индексация: Генерация векторов для всех фрагментов корпуса
print("⏳ Генерация эмбеддингов для веб-корпуса...")
fragment_embeddings_web = create_embeddings(corpus_fragments_web)
print(f"✅ Индексация веб-страницы завершена. Получено {len(fragment_embeddings_web)} векторов.")

⏳ Генерация эмбеддингов для PDF корпуса...
✅ Индексация PDF завершена. Получено 352 векторов.
⏳ Генерация эмбеддингов для веб-корпуса...
✅ Индексация веб-страницы завершена. Получено 13 векторов.


In [11]:
def calculate_cosine_similarity(vec1, vec2):
    """Вычисляет косинусное сходство между двумя векторами NumPy."""
    # np.dot(vec1, vec2) - скалярное произведение
    # np.linalg.norm(vec) - Евклидова норма (длина) вектора
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def retrieve_context(user_query, corpus_fragments, fragment_embeddings, k_neighbors=3):
    """Находит топ-K фрагментов корпуса, наиболее близких к запросу по смыслу.

    :param k_neighbors: Количество самых релевантных фрагментов для извлечения.
    :return: Список из K фрагментов текста.
    """
    # 1. Векторизация запроса
    query_embedding_response = client.embeddings.create(
        input=[user_query],
        model="BAAI/bge-multilingual-gemma2"
    )
    # Преобразуем список чисел в вектор NumPy для расчетов
    query_embedding = np.array(query_embedding_response.data[0].embedding)

    # 2. Расчет сходства
    similarity_scores = []
    for i, emb_data in enumerate(fragment_embeddings):
        fragment_vector = np.array(emb_data.embedding)
        score = calculate_cosine_similarity(query_embedding, fragment_vector)
        similarity_scores.append((i, score))

    # 3. Сортировка и выбор топ-K
    # Сортируем по убыванию (чем выше score, тем лучше)
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    top_k_indices = [index for index, score in similarity_scores[:k_neighbors]]

    return [corpus_fragments[i] for i in top_k_indices]

In [12]:
def generate_augmented_response(query, context_fragments, model_name="Qwen/Qwen3-32B"):
    """Формирует промпт из контекста и запроса и вызывает LLM для ответа."""

    system_prompt = "Вы — экспертный ассистент по работе с документами. Ваш ответ должен быть строго основан на предоставленном КОНТЕКСТЕ. Если в КОНТЕКСТЕ нет информации для полного ответа, вы должны сообщить, что информация не найдена, не выдумывая фактов."

    # Объединяем найденные фрагменты в единый, легко читаемый контекст
    context_text = "\n\n--- РЕЛЕВАНТНЫЙ ФРАГМЕНТ ---\n\n".join(context_fragments)

    full_user_message = f"""
    КОНТЕКСТ ДЛЯ ОТВЕТА:
    {context_text}

    ВОПРОС ПОЛЬЗОВАТЕЛЯ:
    {query}
    """

    response = client.chat.completions.create(
        model=model_name,
        temperature=0, # Обеспечивает фактическую точность и снижает "галлюцинации"
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": full_user_message}
        ]
    )

    return response.choices[0].message.content

In [15]:
def interactive_query_system(corpus_fragments, fragment_embeddings, data_source="загруженному файлу"):
    """Интерактивная система для вопросов-ответов."""

    print(f"\n{'='*60}")
    print(f"ИНТЕРАКТИВНАЯ СИСТЕМА ВОПРОС-ОТВЕТ ({data_source.upper()})")
    print('='*60)

    while True:
        print("\n" + "-"*40)
        user_query = input(f"Введите ваш вопрос по {data_source} (или 'выход' для завершения): ")

        if user_query.lower() in ['выход', 'exit', 'quit', 'q']:
            print("Завершение работы системы...")
            break

        print("\n" + "*"*40)
        print("🔍 Этап 1: Извлечение контекста (Retrieval)")

        # Извлекаем 3 самых релевантных фрагмента
        top_k_fragments = retrieve_context(user_query, corpus_fragments, fragment_embeddings, k_neighbors=3)

        # Выводим найденные фрагменты для проверки прозрачности работы
        for i, chunk in enumerate(top_k_fragments):
            print(f"\n--- Найденный фрагмент {i+1} ---\n{chunk[:200]}...") # Выводим только первые 200 символов

        print("\n" + "*"*40)
        print("🤖 Этап 2: Генерация ответа (Augmentation)")

        # Генерируем финальный ответ
        final_answer = generate_augmented_response(user_query, top_k_fragments)

        print("\n" + "="*60)
        print("ФИНАЛЬНЫЙ ОТВЕТ СИСТЕМЫ RAG:")
        print("="*60)
        print(final_answer)
        print("="*60)

In [19]:
# Основной блок выполнения
def main():
    # Запускаем интерактивную систему для PDF
    interactive_query_system(corpus_fragments_pdf, fragment_embeddings_pdf, "загруженному PDF файлу")

# Запуск основной функции
if __name__ == "__main__":
    main()


ИНТЕРАКТИВНАЯ СИСТЕМА ВОПРОС-ОТВЕТ (ЗАГРУЖЕННОМУ PDF ФАЙЛУ)

----------------------------------------
Введите ваш вопрос по загруженному PDF файлу (или 'выход' для завершения): Расскажи мне про эту модель самсунга

****************************************
🔍 Этап 1: Извлечение контекста (Retrieval)

--- Найденный фрагмент 1 ---
равильного и безопасного использования устройства, прежде чем приступать к работе с ним, изучите это руководство.

- Приведенные ниже описания основаны на настройках устройства по умолчанию.
- Приведе...

--- Найденный фрагмент 2 ---
 Приложения                                       |
|   162 | Общие настройки                                  | Общие настройки                                  |
|   163 | Специальные возможности   ...

--- Найденный фрагмент 3 ---
ля первой настройки и корректной активации всех функций устройства требуется подключение к Wi-Fi или мобильному интернету.

## Обозначения, используемые в данном руководстве

<!-- image -->

Предупреж..

In [20]:
# Основной блок выполнения
def main():
    # Запускаем интерактивную систему для веб-страницы
    interactive_query_system(corpus_fragments_web, fragment_embeddings_web, "веб-странице")

# Запуск основной функции
if __name__ == "__main__":
    main()


ИНТЕРАКТИВНАЯ СИСТЕМА ВОПРОС-ОТВЕТ (ВЕБ-СТРАНИЦЕ)

----------------------------------------
Введите ваш вопрос по веб-странице (или 'выход' для завершения): Что ты знаешь про Figure 02?

****************************************
🔍 Этап 1: Извлечение контекста (Retrieval)

--- Найденный фрагмент 1 ---
оботы стали быстрее и лучше справлялись с задачами на производстве.
figure ai
figure
f.02
figure 02
роботы
bmw
гуманоидные роботы
человекоподобные роботы
Робототехника
Транспорт
Будущее здесь
Развитие...

--- Найденный фрагмент 2 ---
обных роботов Figure 02 (F.02) на заводе BMW в штате Южная Каролина. Стартап подвёл итоги развёртывания Figure 02 на фоне снятия модели с производства.
Пилотный проект был частью совместной работы ком...

--- Найденный фрагмент 3 ---
Figure AI завершила почти годичные испытания человекоподобных роботов Figure 02 на заводе BMW в Южной Каролине
Калифорнийский стартап Figure AI отчитался о завершении 11-месячных испытаний человекопод...

*************************